In [1]:
%pip install datasets
%pip install sentencepiece
%pip install transformers
%pip install accelerate -U
%pip install transformers[torch]
%pip install evaluate
%pip install rouge_score
%pip install indic-nlp-library
%pip install nltk

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

In [2]:
from datasets import Dataset
from datasets import load_dataset
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
import re
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration
import numpy as np
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/poornaditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dataset_gujarati = load_dataset("csebuetnlp/xlsum", "gujarati")
dataset_hindi = load_dataset("csebuetnlp/xlsum","hindi")
dataset_telugu = load_dataset("csebuetnlp/xlsum","telugu")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [14]:
train_text_hi = dataset_hindi['train']['text'][:5000]
train_summary_hi = dataset_hindi['train']['summary'][:5000]

test_text_hi = dataset_hindi['test']['text'][:1200]
test_summary_hi = dataset_hindi['test']['summary'][:1200]

eval_text_hi = dataset_hindi['validation']['text'][:1200]
eval_summary_hi = dataset_hindi['validation']['summary'][:1200]

In [15]:
train_text_gj = dataset_gujarati['train']['text']
train_summary_gj = dataset_gujarati['train']['summary']

test_text_gj = dataset_gujarati['test']['text']
test_summary_gj = dataset_gujarati['test']['summary']

eval_text_gj = dataset_gujarati['validation']['text']
eval_summary_gj = dataset_gujarati['validation']['summary']

In [16]:
train_text_te = dataset_telugu['train']['text']
train_summary_te = dataset_telugu['train']['summary']

test_text_te = dataset_telugu['test']['text']
test_summary_te = dataset_telugu['test']['summary']

eval_text_te = dataset_telugu['validation']['text']
eval_summary_te = dataset_telugu['validation']['summary']

In [ ]:
# train_dataset = Dataset.from_dict({"text": train_text, "summary": train_summary})
# test_dataset = Dataset.from_dict({"text": test_text, "summary": test_summary})

# eval_dataset = Dataset.from_dict({"text": eval_text, "summary": eval_summary})

In [21]:
def clean_gujarati_text(text):
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("gu")
    cleaned_text = re.sub(r'[^\u0A80-\u0AFF0-9\s]', '', text)
    cleaned_text = normalizer.normalize(cleaned_text)

    return cleaned_text

def clean_hindi_text(text):
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("hi")
    cleaned_text = re.sub(r'[^\u0900-\u097F0-9\s]', '', text)
    cleaned_text = normalizer.normalize(cleaned_text)

    return cleaned_text

def clean_telugu_text(text):
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("te")
    cleaned_text = re.sub(r'[^\u0C00-\u0C7F0-9\s]', '', text)
    cleaned_text = normalizer.normalize(cleaned_text)
    
    return cleaned_text

In [22]:
import evaluate

rouge = evaluate.load("rouge")

In [23]:
cleaned_train_text_hi = [clean_hindi_text(text) for text in train_text_hi]
cleaned_train_summary_hi = [clean_hindi_text(text) for text in train_summary_hi]

cleaned_test_text_hi = [clean_hindi_text(text) for text in test_text_hi]
cleaned_test_summary_hi = [clean_hindi_text(text) for text in test_summary_hi]

cleaned_eval_text_hi = [clean_hindi_text(text) for text in eval_text_hi]
cleaned_eval_summary_hi = [clean_hindi_text(text) for text in eval_summary_hi]

In [24]:
cleaned_train_text_gj = [clean_gujarati_text(text) for text in train_text_gj]
cleaned_train_summary_gj = [clean_gujarati_text(text) for text in train_summary_gj]

cleaned_test_text_gj = [clean_gujarati_text(text) for text in test_text_gj]
cleaned_test_summary_gj = [clean_gujarati_text(text) for text in test_summary_gj]

cleaned_eval_text_gj = [clean_gujarati_text(text) for text in eval_text_gj]
cleaned_eval_summary_gj = [clean_gujarati_text(text) for text in eval_summary_gj]

In [25]:
cleaned_train_text_te = [clean_telugu_text(text) for text in train_text_te]
cleaned_train_summary_te = [clean_telugu_text(text) for text in train_summary_te]

cleaned_test_text_te = [clean_telugu_text(text) for text in test_text_te]
cleaned_test_summary_te = [clean_telugu_text(text) for text in test_summary_te]

cleaned_eval_text_te = [clean_telugu_text(text) for text in eval_text_te]
cleaned_eval_summary_te = [clean_telugu_text(text) for text in eval_summary_te]

In [32]:
train_text = cleaned_train_text_hi + cleaned_train_text_gj + cleaned_train_text_te
train_summary = cleaned_train_summary_hi + cleaned_train_summary_gj + cleaned_train_summary_te

test_text = cleaned_test_text_hi + cleaned_test_text_gj + cleaned_test_text_te
test_summary = cleaned_test_summary_hi + cleaned_test_summary_gj + cleaned_test_summary_te

eval_text = cleaned_eval_text_hi + cleaned_eval_text_gj + cleaned_eval_text_te
eval_summary = cleaned_eval_summary_hi + cleaned_eval_summary_gj + cleaned_eval_summary_te

In [33]:
train_dataset = Dataset.from_dict({"text": train_text, "summary": train_summary})
test_dataset = Dataset.from_dict({"text": test_text, "summary": test_summary})
eval_dataset = Dataset.from_dict({"text": eval_text, "summary": eval_summary})

In [ ]:
# cleaned_train_text = [clean_gujarati_text(text) for text in train_text]
# cleaned_train_summary = [clean_gujarati_text(summary) for summary in train_summary]
# cleaned_test_text = [clean_gujarati_text(text) for text in test_text]
# cleaned_test_summary = [clean_gujarati_text(summary) for summary in test_summary]
# cleaned_eval_text = [clean_gujarati_text(text) for text in eval_text]
# cleaned_eval_summary = [clean_gujarati_text(summary) for summary in eval_summary]

# train_dataset = Dataset.from_dict({"text": cleaned_train_text, "summary": cleaned_train_summary})
# test_dataset = Dataset.from_dict({"text": cleaned_test_text, "summary": cleaned_test_summary})
# eval_dataset = Dataset.from_dict({"text": cleaned_eval_text, "summary": cleaned_eval_summary})

In [34]:
from transformers import AutoTokenizer

model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# cleaned_train_text = [clean_gujarati_text(text) for text in train_text]
# cleaned_train_summary = [clean_gujarati_text(summary) for summary in train_summary]
# cleaned_test_text = [clean_gujarati_text(text) for text in test_text]
# cleaned_test_summary = [clean_gujarati_text(summary) for summary in test_summary]
# cleaned_eval_text = [clean_gujarati_text(text) for text in eval_text]
# cleaned_eval_summary = [clean_gujarati_text(summary) for summary in eval_summary]

# train_dataset = Dataset.from_dict({"text": cleaned_train_text, "summary": cleaned_train_summary})
# test_dataset = Dataset.from_dict({"text": cleaned_test_text, "summary": cleaned_test_summary})
# eval_dataset = Dataset.from_dict({"text": cleaned_eval_text, "summary": cleaned_eval_summary})

In [35]:
prefix = "summarize: "
max_input_length = 512
max_target_length = 64

def preprocess_data(data):
  texts_cleaned = data["text"]
  inputs = [prefix + text for text in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(data["summary"], max_length=max_target_length,
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [36]:
tokenized_train_dataset = train_dataset.map(preprocess_data,
                                            batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_data,
                                          batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_data,
                                          batched=True)

Map:   0%|          | 0/24540 [00:00<?, ? examples/s]

/Users/poornaditya/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3641 [00:00<?, ? examples/s]

Map:   0%|          | 0/3641 [00:00<?, ? examples/s]

In [37]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [39]:
batch_size = 2
model_name = "t5-base-medium-title-generation"
model_dir = "output_dir"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard",
)

In [40]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [41]:
from datasets import load_metric

metric = load_metric("rouge")

/var/folders/rv/d60y2tpx6jjcycjgzs_yj0_80000gn/T/ipykernel_37541/2412581395.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [42]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:


model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

trainer = Seq2SeqTrainer(
    # model_init=model_init,
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# trainer.args.device = device  # Set Trainer device

# Start training
trainer.train()

In [47]:
model = T5ForConditionalGeneration.from_pretrained('output_dir/..')


OSError: It looks like the config file at 'output_dir/runs/Dec10_18-21-05_Poornadityas-MacBook-Pro.local/events.out.tfevents.1702250530.Poornadityas-MacBook-Pro.local.37541.2' is not a valid JSON file.

In [48]:
def generate_summary(input_text, max_length=150):
    inputs = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs.to(model.device), max_length=max_length, num_beams=2, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
input_test = eval_dataset['text'][0]

genereated_summary = generate_summary(input_test)
print(genereated_summary)

In [ ]:
print("Original summary:", eval_dataset['summary'][0]) 